## Data collection and imputation

In [61]:
%load_ext autoreload
%autoreload 2

In [46]:
import pandas as pd

In [10]:
df = pd.read_excel('2022-offense.xlsx', sheet_name='Offense_Prior_Actuals', header=3)

# Drop columns that we don't need
drop_columns = ['GS', 'Bye', 'Notes', 'Rank', 'Y! Roto', 'Δ', 'Y! Old', 'Std', 'Δ.1', 'Std Old', 'PPR', 'Δ.2', 'PPR Old', '% Own', 'Fan Pts', 'PPG']
df = df.drop(drop_columns, axis=1)

# Change names of duplicate columns
updated_columns = {'Yds': 'Passing_Yds', 'Yds.1': 'Rushing_Yds', 'Yds.2': 'Receiving_Yds', 'Yds.3': 'Return_Yds', 
                   'TD': 'Passing_Td', 'TD.1': 'Rushing_Td', 'TD.2': 'Receiving_Td', 'TD.3': 'Return_Td', 
                   '1st': 'Passing_1st', '1st.1': 'Rushing_1st', '1st.2': 'Receiving_1st'}
df = df.rename(columns=updated_columns)


Offense_Prior_Actuals
Offense_Proj


/Users/ethancoomber/opt/anaconda3/envs/ml-0451/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from unicodedata import normalize

table_contracts = pd.read_html('https://www.spotrac.com/nfl/contracts/sort-value/offense/limit-300/')

In [40]:
player_contracts = table_contracts[0]
player_contracts

,Rank,Player,Signed Age,Yrs,Value,AAV,Sign Bonus,G'teed @ Sign,Practical G'teed
0,1,Patrick Mahomes Quarterback | 2020-2031 (FA: ...,24,10,"$450,000,000","$45,000,000","$10,000,000","$63,081,905","$141,481,905"
1,2,Josh Allen Quarterback | 2021-2028 (FA: 2029),25,6,"$258,034,000","$43,005,667","$16,500,000","$100,038,596","$150,000,000"
2,3,Jalen Hurts Quarterback | 2023-2028 (FA: 2029),24,5,"$255,000,000","$51,000,000","$23,294,000","$110,000,000","$179,300,000"
3,4,Russell Wilson Quarterback | 2022-2028 (FA: 2...,33,5,"$242,588,236","$48,517,647","$50,000,000","$124,000,000","$161,000,000"
4,5,Kyler Murray Quarterback | 2022-2028 (FA: 2029),24,5,"$230,500,000","$46,100,000","$29,035,000","$103,300,000","$159,797,000"
...,...,...,...,...,...,...,...,...,...
295,296,Luke Fortner Center | 2022-2025 (FA: 2026),24,4,"$5,500,065","$1,375,016","$1,180,048","$1,180,048","$1,180,048"
296,297,Joshua Ezeudu Guard | 2022-2025 (FA: 2026),22,4,"$5,485,524","$1,371,381","$1,169,472","$1,169,472","$1,169,472"
297,298,Denzel Mims Wide Receiver | 2020-2023 (FA: 2024),22,4,"$5,433,700","$1,358,425","$1,511,784","$2,368,770","$2,368,770"
298,299,Velus Jones Wide Receiver | 2022-2025 (FA: 2026),25,4,"$5,405,928","$1,351,482","$1,111,584","$1,111,584","$1,111,584"


In [48]:
# Get y - actual values from the next year (we do not have 2023 but could get them) - we will eventually need them?
QB_data = df[df["Pos"] == "QB"]

# Get the list of all column names from headers
qb_column_headers = list(QB_data.columns.values)
# print("The Column Header :", qb_column_headers)

qb_drop_columns = ['Tgt', 'Rec', 'Receiving_Yds', 'Receiving_Td', 'Receiving_1st', 'Return_Yds', 'Return_Td', '2PT',]
QB_data = QB_data.drop(qb_drop_columns, axis=1)
# QB_data



In [68]:
from fantasy_football import Fantasy

football = Fantasy()
football.addContracts(QB_data, player_contracts)

             Player Pos Team  GP  Comp  Inc  Passing_Yds  Passing_Td  Int  \
0          J. Allen  QB  Buf  17   409  237         4407          36   15   
1        J. Herbert  QB  LAC  17   443  229         5014          38   15   
2        P. Mahomes  QB   KC  17   436  222         4839          37   13   
3          T. Brady  QB   TB  17   485  234         5316          43   12   
4         J. Burrow  QB  Cin  16   366  154         4611          34   14   
5       M. Stafford  QB  LAR  17   404  197         4886          41   17   
6       D. Prescott  QB  Dal  16   410  186         4449          37   10   
7        A. Rodgers  QB   GB  16   366  165         4115          37    4   
8      R. Tannehill  QB  Ten  17   357  174         3734          21   14   
9           D. Carr  QB   LV  17   428  198         4804          23   14   
10         J. Hurts  QB  Phi  15   265  167         3144          16    9   
11        K. Murray  QB  Ari  14   333  148         3787          24   10   

AttributeError: 'DataFrame' object has no attribute 'rows'

In [24]:
RB_data = df[df["Pos"] == "RB"]

rb_drop_columns = [ 'Comp', 'Inc', 'Passing_Yds', 'Passing_Td', 'Int', 'Pic6', 'Sks', 'Passing_1st']
RB_data = RB_data.drop(rb_drop_columns, axis=1)
# RB_data

In [23]:
WR_data = df[df["Pos"] == "WR"]

wr_drop_columns = ['Comp', 'Inc', 'Passing_Yds', 'Passing_Td', 'Int', 'Pic6', 'Sks', 'Passing_1st']
WR_data = WR_data.drop(rb_drop_columns, axis=1)
# WR_data

In [25]:
TE_data = df[df["Pos"] == "TE"]

te_drop_columns = ['Comp', 'Inc', 'Passing_Yds', 'Passing_Td', 'Int', 'Pic6', 'Sks', 'Passing_1st', 'Return_Yds', 'Return_Td']
TE_data = TE_data.drop(te_drop_columns, axis=1)
TE_data

,Player,Pos,Team,GP,Att,Rushing_Yds,Rushing_Td,Rushing_1st,Tgt,Rec,Receiving_Yds,Receiving_Td,Receiving_1st,2PT,Total,Lost
45,M. Andrews,TE,Bal,17,1,0,0,0,153,107,1361,9,75,2,1,0
62,T. Kelce,TE,KC,16,2,3,1,1,134,92,1125,9,63,0,1,1
96,D. Schultz,TE,Dal,17,0,0,0,0,104,78,808,8,43,1,0,0
98,G. Kittle,TE,SF,14,3,20,0,0,94,71,910,6,42,0,2,1
111,K. Pitts,TE,Atl,17,0,0,0,0,110,68,1026,1,43,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,J. Akins,TE,NYG,13,1,3,0,1,33,24,214,0,11,0,2,2
356,Z. Gentry,TE,Pit,17,0,0,0,0,25,19,167,0,11,0,0,0
358,E. Ebron,TE,Pit,8,1,1,1,1,18,12,84,1,6,0,0,0
365,N. Vannett,TE,NO,7,0,0,0,0,15,9,133,1,7,0,0,0
